## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-02-16-52-15 +0000,nyt,"Synagogue Attack on Yom Kippur in Manchester, ...",https://www.nytimes.com/2025/10/02/world/europ...
1,2025-10-02-16-51-29 +0000,nyt,Israel Intercepted the Flotilla Headed to Gaza...,https://www.nytimes.com/2025/10/01/world/middl...
2,2025-10-02-16-50-35 +0000,nyt,Live Updates: British Police Say Yom Kippur At...,https://www.nytimes.com/live/2025/10/02/world/...
3,2025-10-02-16-50-24 +0000,nyt,Madagascar’s President Rajoelina Dissolved His...,https://www.nytimes.com/2025/10/02/world/afric...
4,2025-10-02-16-50-23 +0000,nyt,Energy Dept. Cancels Hundreds of Clean-Energy ...,https://www.nytimes.com/2025/10/02/climate/doe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
127,shutdown,37
36,trump,34
21,government,25
121,new,17
10,gaza,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
21,2025-10-02-16-28-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...,148
241,2025-10-01-22-43-00 +0000,wsj,Many GOP senators seeking an end to the govern...,https://www.wsj.com/politics/policy/white-hous...,134
247,2025-10-01-22-10-12 +0000,nypost,Trump admin will keep Statue of Liberty open d...,https://nypost.com/2025/10/01/us-news/trump-ad...,120
221,2025-10-02-00-46-00 +0000,wsj,Congress’s decision about what to do about exp...,https://www.wsj.com/politics/policy/government...,108
99,2025-10-02-13-17-00 +0000,wsj,Office of Management and Budget Director Russe...,https://www.wsj.com/politics/policy/russell-vo...,107


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
21,148,2025-10-02-16-28-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...
338,68,2025-10-01-17-43-30 +0000,nypost,What will ‘TrumpRx’ actually mean for patients...,https://nypost.com/2025/10/01/health/what-will...
233,60,2025-10-01-23-07-36 +0000,wapo,Israel steps up Gaza City assault as Hamas wei...,https://www.washingtonpost.com/world/2025/10/0...
145,57,2025-10-02-10-09-41 +0000,cbc,U.K. police say deadly rampage at Manchester s...,https://www.cbc.ca/news/world/uk-synagogue-inc...
225,54,2025-10-02-00-30-00 +0000,wsj,The Court kept Lisa Cook in her job until it c...,https://www.wsj.com/politics/policy/the-suprem...
232,49,2025-10-01-23-09-00 +0000,wsj,The Trump administration said it is withholdin...,https://www.wsj.com/politics/policy/trump-admi...
335,41,2025-10-01-18-00-00 +0000,latimes,"As California glaciers disappear, people will ...",https://www.latimes.com/environment/story/2025...
221,38,2025-10-02-00-46-00 +0000,wsj,Congress’s decision about what to do about exp...,https://www.wsj.com/politics/policy/government...
300,36,2025-10-01-19-43-41 +0000,cbc,"Scientists, stars, statespeople mourn Jane Goo...",https://www.cbc.ca/news/world/jane-goodall-dea...
184,36,2025-10-02-05-15-26 +0000,nypost,Trump admin to provide Ukraine with intelligen...,https://nypost.com/2025/10/02/us-news/trump-ad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
